In [52]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time
from datetime import datetime, timedelta
import pandas as pd
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData
import cx_Oracle
from sqlalchemy import create_engine
import seaborn as sns
sns.set_style('whitegrid')
import pyecharts as pye
from pyecharts import options as opts
from pyecharts.charts import Line
from pyecharts.charts import Timeline, Bar, Line ,Grid
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from IPython.display import IFrame
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sqlalchemy.dialects.oracle import NUMBER, VARCHAR2
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, Text

In [26]:
db_username = 'base_man'
db_password = '1111'
db_host = '192.168.0.29' # 다른 IP 주소
db_port = '1521' #Oracle의 기본포트
service_name = 'XE'

# Oracle 데이터베이스 연결 문자열 생성
dsn_tns = cx_Oracle.makedsn(db_host, db_port, service_name=service_name)

# 데이터베이스 연결
# connection = cx_Oracle.connect(user=db_username, password=db_password, dsn=dsn_tns)

pool = cx_Oracle.SessionPool(user=db_username, password=db_password, dsn=dsn_tns,
                             min=2, max=5, increment=1, threaded=True)

# 커넥션 풀에서 연결 얻기
connection = pool.acquire()

# SQL 쿼리 작성
query = 'SELECT * FROM youtube'

# 쿼리 실행 및 데이터프레임 생성
try:   
    reply_pd = pd.read_sql(query, con=connection)
    # '댓글내용' 열이 CLOB 타입일 경우 문자열로 변환
    if '댓글내용' in reply_pd.columns:
        reply_pd['댓글내용'] = reply_pd['댓글내용'].apply(lambda x: str(x) if x is not None else '')

    print(reply_pd.head())
finally:
    # 커넥션 반납
    pool.release(connection)

# 커넥션 풀 종료
pool.close()

                                                  제목  \
0  하늘엔 수많은 색깔의 별이 있잖아. 그 중에 가장 뜨거운 빛을 뿜어 내는 건, 파랑...   
1  ✨삼성라이온즈✨ 평점 10/10 도파민 터지는 삼성라이온즈 야구를 추천하오. 소인은...   
2        8월 3번째 스윕승, 4연승, 승리 직후의 모습#애프터매치(08.29 키움전)   
3       WIN and WOW, 3연승, 승리 직후의 모습#애프터매치(08.28 키움전)   
4     데뷔 첫 완봉승, 천금의 한 점, 승리 직후의 모습 #애프터매치(08.27 키움전)   

                                                댓글내용  \
0  “ 유격수 이재현이 더이상의 실점을 허락하지 않습니다 ” 정우영 캐스터가 중계한 멘...   
1  정빈이는 진짜 유명한 ㅈ댐감지기가 맞다 ㅋㅋㅋㅋㅋㅋㅋ진짜 벼랑끝에 왔을때 하나씩 해...   
2  우리의 지금 순위는 구자욱이 주장인게 진짜 의미가 있다... 잘하는 선수 바로바로 ...   
3  아니이제 애프터매치도 낯안가리는 박뱅이라니...... 귀하다귀해 인터뷰 안 피하는 ...   
4  갑작스럽게 나오게 되었지만 늘 꼭 필요한 순간에 한방 터트려주는 안경 선배 멋있어요...   

                                            링크  
0                                         None  
1                                         None  
2  https://www.youtube.com/watch?v=4CEvMrPw5xw  
3                                         None  
4                                         None  


In [27]:
reply_pd.head()

,제목,댓글내용,링크
0,"하늘엔 수많은 색깔의 별이 있잖아. 그 중에 가장 뜨거운 빛을 뿜어 내는 건, 파랑...",“ 유격수 이재현이 더이상의 실점을 허락하지 않습니다 ” 정우영 캐스터가 중계한 멘...,None
1,✨삼성라이온즈✨ 평점 10/10 도파민 터지는 삼성라이온즈 야구를 추천하오. 소인은...,정빈이는 진짜 유명한 ㅈ댐감지기가 맞다 ㅋㅋㅋㅋㅋㅋㅋ진짜 벼랑끝에 왔을때 하나씩 해...,None
2,"8월 3번째 스윕승, 4연승, 승리 직후의 모습#애프터매치(08.29 키움전)",우리의 지금 순위는 구자욱이 주장인게 진짜 의미가 있다... 잘하는 선수 바로바로 ...,https://www.youtube.com/watch?v=4CEvMrPw5xw
3,"WIN and WOW, 3연승, 승리 직후의 모습#애프터매치(08.28 키움전)",아니이제 애프터매치도 낯안가리는 박뱅이라니...... 귀하다귀해 인터뷰 안 피하는 ...,None
4,"데뷔 첫 완봉승, 천금의 한 점, 승리 직후의 모습 #애프터매치(08.27 키움전)",갑작스럽게 나오게 되었지만 늘 꼭 필요한 순간에 한방 터트려주는 안경 선배 멋있어요...,None


In [28]:
df = reply_pd.drop('링크', axis=1)

In [29]:
df

,제목,댓글내용
0,"하늘엔 수많은 색깔의 별이 있잖아. 그 중에 가장 뜨거운 빛을 뿜어 내는 건, 파랑...",“ 유격수 이재현이 더이상의 실점을 허락하지 않습니다 ” 정우영 캐스터가 중계한 멘...
1,✨삼성라이온즈✨ 평점 10/10 도파민 터지는 삼성라이온즈 야구를 추천하오. 소인은...,정빈이는 진짜 유명한 ㅈ댐감지기가 맞다 ㅋㅋㅋㅋㅋㅋㅋ진짜 벼랑끝에 왔을때 하나씩 해...
2,"8월 3번째 스윕승, 4연승, 승리 직후의 모습#애프터매치(08.29 키움전)",우리의 지금 순위는 구자욱이 주장인게 진짜 의미가 있다... 잘하는 선수 바로바로 ...
3,"WIN and WOW, 3연승, 승리 직후의 모습#애프터매치(08.28 키움전)",아니이제 애프터매치도 낯안가리는 박뱅이라니...... 귀하다귀해 인터뷰 안 피하는 ...
4,"데뷔 첫 완봉승, 천금의 한 점, 승리 직후의 모습 #애프터매치(08.27 키움전)",갑작스럽게 나오게 되었지만 늘 꼭 필요한 순간에 한방 터트려주는 안경 선배 멋있어요...
5,영원히 야구장에 푸른 파도가 멈추지 않도록 📹 #덕아웃1열 (08.29 키움전),"아마도 22년 2월. 구자욱의 블로그에서 감동적으로 읽고 기억해 두었던 그 글귀 ""..."
6,(월간라클) 팬 vs 선수 라이온즈 푸른 피 TEST✏ #월간라클,2:12 좌승보다 지찬이가 형이라니...깜짝놀랐네.. 1:23 영웅이 협찬 해달라는...
7,"""인성이 여기서 나타나는 거야"" ""우리 지광이는요..평양냉면을 좋아해요"" T와 F적...",지찬아 너 삼성이랑 20년 계약했어… 기억 안 나 ? 태인아 너 비fa다년계약 20...


In [30]:
base_sentiment_weights = {
    '좋아요': 1.0, '재미있어요': 1.0, '추천': 1.0,
    '별로에요': -1.0, '지루해요': -1.0, '실망': -1.0,
    '정말': 0.5, '매우': 0.5, '아주': 0.5,
    '조금': -0.5, '별로': -0.5, '최고': 1.0,
    '행복': 1.0, '기쁘다': 1.0, '뛰어나다': 1.0,
    '감동': 1.0, '만족': 1.0, '사랑': 1.0,
    '흥미진진': 1.0, '훌륭하다': 1.0, '귀엽다': 1.0,
    '반갑다': 1.0, '신나다': 1.0, '아름답다': 1.0,
    '최악': -1.0, '불편하다': -1.0, '싫다': -1.0,
    '불만': -1.0, '아쉽다': -1.0, '문제': -1.0,
    '피곤': -1.0, '지루하다': -1.0, '나쁘다': -1.0,
    '심하다': -1.0, '위험': -1.0, '비싸다': -1.0,
    '쓸데없다': -1.0, '화나다': -1.0, '졸리다': -1.0,
    '형편없다': -1.0, '혼란스럽다': -1.0, '엉망': -1.0,
    '실패': -1.0, '힘들다': -1.0, '안타깝다': -1.0,
    '무서운': -1.0, '위험한': -1.0, '억울하다': -1.0,
    '소름끼치다': -1.0, '불쾌하다': -1.0, '안되다': -1.0
}

baseball_sentiment_weights = {
    '홈런' : 1.0, '안타' : 1.0 , '실점' : -1.0,
    '부상' : -2.0, '메이저' : 1.0, '득점' : 1.0,
    '빈볼' : -1.0, '약속' : 1.0, '화이팅' :1.0,
    '호수비' : 1.0, '도루' : 1.0, '타점' : 1.0,
    '피안타' : -1.0, '피홈런' : -1.0, '실책' : -1.0,
    '우리' : 1.0, '연승' : 1.0, '연패' :-2.0,
    '우승' : 1.0, '에이스' : 1.0, '민폐' : -1.0,
    '스윕' : 1.0, '역스윕' : -1.0, '홀드' :1.0,
    '병살' :-1.0, '다이빙캐치' :1.0
}
# 두 사전을 합칩니다
combined_sentiment_weights = base_sentiment_weights.copy()
combined_sentiment_weights.update(baseball_sentiment_weights)

In [31]:
def analyze_sentiment(text):
    okt = Okt()
    tokens = okt.pos(text, stem=True)
    
    sentiment_score = 0
    for token, pos in tokens:
        if pos in ['Noun', 'Adjective', 'Adverb'] and token in combined_sentiment_weights:
            sentiment_score += combined_sentiment_weights[token]
    
    # 0> 긍정적, 0<: 부정적, 0: 중립적
    return ("긍정적" if sentiment_score > 0 else "부정적" if sentiment_score < 0 else "중립적" ,sentiment_score)

In [32]:
df['감성분석'] = df['댓글내용'].apply(lambda x: analyze_sentiment(x)[0])
df['감성점수'] = df['댓글내용'].apply(lambda x: analyze_sentiment(x)[1])

In [33]:
df

,제목,댓글내용,감성분석,감성점수
0,"하늘엔 수많은 색깔의 별이 있잖아. 그 중에 가장 뜨거운 빛을 뿜어 내는 건, 파랑...",“ 유격수 이재현이 더이상의 실점을 허락하지 않습니다 ” 정우영 캐스터가 중계한 멘...,긍정적,124.0
1,✨삼성라이온즈✨ 평점 10/10 도파민 터지는 삼성라이온즈 야구를 추천하오. 소인은...,정빈이는 진짜 유명한 ㅈ댐감지기가 맞다 ㅋㅋㅋㅋㅋㅋㅋ진짜 벼랑끝에 왔을때 하나씩 해...,긍정적,36.5
2,"8월 3번째 스윕승, 4연승, 승리 직후의 모습#애프터매치(08.29 키움전)",우리의 지금 순위는 구자욱이 주장인게 진짜 의미가 있다... 잘하는 선수 바로바로 ...,긍정적,126.0
3,"WIN and WOW, 3연승, 승리 직후의 모습#애프터매치(08.28 키움전)",아니이제 애프터매치도 낯안가리는 박뱅이라니...... 귀하다귀해 인터뷰 안 피하는 ...,긍정적,74.5
4,"데뷔 첫 완봉승, 천금의 한 점, 승리 직후의 모습 #애프터매치(08.27 키움전)",갑작스럽게 나오게 되었지만 늘 꼭 필요한 순간에 한방 터트려주는 안경 선배 멋있어요...,긍정적,53.0
5,영원히 야구장에 푸른 파도가 멈추지 않도록 📹 #덕아웃1열 (08.29 키움전),"아마도 22년 2월. 구자욱의 블로그에서 감동적으로 읽고 기억해 두었던 그 글귀 ""...",긍정적,94.0
6,(월간라클) 팬 vs 선수 라이온즈 푸른 피 TEST✏ #월간라클,2:12 좌승보다 지찬이가 형이라니...깜짝놀랐네.. 1:23 영웅이 협찬 해달라는...,긍정적,19.0
7,"""인성이 여기서 나타나는 거야"" ""우리 지광이는요..평양냉면을 좋아해요"" T와 F적...",지찬아 너 삼성이랑 20년 계약했어… 기억 안 나 ? 태인아 너 비fa다년계약 20...,긍정적,21.5


In [34]:
df_sorted = df.sort_values(by='감성점수', ascending=False)
df_sorted

,제목,댓글내용,감성분석,감성점수
2,"8월 3번째 스윕승, 4연승, 승리 직후의 모습#애프터매치(08.29 키움전)",우리의 지금 순위는 구자욱이 주장인게 진짜 의미가 있다... 잘하는 선수 바로바로 ...,긍정적,126.0
0,"하늘엔 수많은 색깔의 별이 있잖아. 그 중에 가장 뜨거운 빛을 뿜어 내는 건, 파랑...",“ 유격수 이재현이 더이상의 실점을 허락하지 않습니다 ” 정우영 캐스터가 중계한 멘...,긍정적,124.0
5,영원히 야구장에 푸른 파도가 멈추지 않도록 📹 #덕아웃1열 (08.29 키움전),"아마도 22년 2월. 구자욱의 블로그에서 감동적으로 읽고 기억해 두었던 그 글귀 ""...",긍정적,94.0
3,"WIN and WOW, 3연승, 승리 직후의 모습#애프터매치(08.28 키움전)",아니이제 애프터매치도 낯안가리는 박뱅이라니...... 귀하다귀해 인터뷰 안 피하는 ...,긍정적,74.5
4,"데뷔 첫 완봉승, 천금의 한 점, 승리 직후의 모습 #애프터매치(08.27 키움전)",갑작스럽게 나오게 되었지만 늘 꼭 필요한 순간에 한방 터트려주는 안경 선배 멋있어요...,긍정적,53.0
1,✨삼성라이온즈✨ 평점 10/10 도파민 터지는 삼성라이온즈 야구를 추천하오. 소인은...,정빈이는 진짜 유명한 ㅈ댐감지기가 맞다 ㅋㅋㅋㅋㅋㅋㅋ진짜 벼랑끝에 왔을때 하나씩 해...,긍정적,36.5
7,"""인성이 여기서 나타나는 거야"" ""우리 지광이는요..평양냉면을 좋아해요"" T와 F적...",지찬아 너 삼성이랑 20년 계약했어… 기억 안 나 ? 태인아 너 비fa다년계약 20...,긍정적,21.5
6,(월간라클) 팬 vs 선수 라이온즈 푸른 피 TEST✏ #월간라클,2:12 좌승보다 지찬이가 형이라니...깜짝놀랐네.. 1:23 영웅이 협찬 해달라는...,긍정적,19.0


In [40]:
username = 'base_man'
password = '1111'
hostname = '192.168.0.29'
port = '1521'
service_name = 'XE'

# 연결 문자열 작성
dsn = cx_Oracle.makedsn(hostname, port, service_name=service_name)
connection_string = f'oracle+cx_oracle://{username}:{password}@{dsn}'

# SQLAlchemy 엔진 생성
engine = create_engine(connection_string)
query = "SELECT * FROM youtube2"

# 데이터베이스에서 데이터프레임으로 읽기
df = pd.read_sql(query, con=engine)
df

,제목,링크,썸네일
0,"8월 3번째 스윕승, 4연승, 승리 직후의 모습#애프터매치(08.29 키움전)",https://www.youtube.com/watch?v=4CEvMrPw5xw,https://img.youtube.com/vi/4CEvMrPw5xw/maxresd...
1,"WIN and WOW, 3연승, 승리 직후의 모습#애프터매치(08.28 키움전)",https://www.youtube.com/watch?v=PWil5aTn7e4,https://img.youtube.com/vi/PWil5aTn7e4/maxresd...
2,"""인성이 여기서 나타나는 거야"" ""우리 지광이는요..평양냉면을 좋아해요"" T와 F적...",https://www.youtube.com/watch?v=NjrZdLiEHYs,https://img.youtube.com/vi/NjrZdLiEHYs/maxresd...
3,✨삼성라이온즈✨ 평점 10/10 도파민 터지는 삼성라이온즈 야구를 추천하오. 소인은...,https://www.youtube.com/watch?v=qergpNg580U,https://img.youtube.com/vi/qergpNg580U/maxresd...
4,"데뷔 첫 완봉승, 천금의 한 점, 승리 직후의 모습 #애프터매치(08.27 키움전)",https://www.youtube.com/watch?v=QkMRf7lvbe0,https://img.youtube.com/vi/QkMRf7lvbe0/maxresd...
5,"하늘엔 수많은 색깔의 별이 있잖아. 그 중에 가장 뜨거운 빛을 뿜어 내는 건, 파랑...",https://www.youtube.com/watch?v=Qic3h--TfRA,https://img.youtube.com/vi/Qic3h--TfRA/maxresd...
6,영원히 야구장에 푸른 파도가 멈추지 않도록 📹 #덕아웃1열 (08.29 키움전),https://www.youtube.com/watch?v=lgBk9wbil40,https://img.youtube.com/vi/lgBk9wbil40/maxresd...
7,(월간라클) 팬 vs 선수 라이온즈 푸른 피 TEST✏ #월간라클,https://www.youtube.com/watch?v=BRnQTmWaV_A,https://img.youtube.com/vi/BRnQTmWaV_A/maxresd...


In [41]:
merged_df = pd.merge(df_sorted, df, on='제목', how='left')

In [42]:
merged_df

,제목,댓글내용,감성분석,감성점수,링크,썸네일
0,"8월 3번째 스윕승, 4연승, 승리 직후의 모습#애프터매치(08.29 키움전)",우리의 지금 순위는 구자욱이 주장인게 진짜 의미가 있다... 잘하는 선수 바로바로 ...,긍정적,126.0,https://www.youtube.com/watch?v=4CEvMrPw5xw,https://img.youtube.com/vi/4CEvMrPw5xw/maxresd...
1,"하늘엔 수많은 색깔의 별이 있잖아. 그 중에 가장 뜨거운 빛을 뿜어 내는 건, 파랑...",“ 유격수 이재현이 더이상의 실점을 허락하지 않습니다 ” 정우영 캐스터가 중계한 멘...,긍정적,124.0,https://www.youtube.com/watch?v=Qic3h--TfRA,https://img.youtube.com/vi/Qic3h--TfRA/maxresd...
2,영원히 야구장에 푸른 파도가 멈추지 않도록 📹 #덕아웃1열 (08.29 키움전),"아마도 22년 2월. 구자욱의 블로그에서 감동적으로 읽고 기억해 두었던 그 글귀 ""...",긍정적,94.0,https://www.youtube.com/watch?v=lgBk9wbil40,https://img.youtube.com/vi/lgBk9wbil40/maxresd...
3,"WIN and WOW, 3연승, 승리 직후의 모습#애프터매치(08.28 키움전)",아니이제 애프터매치도 낯안가리는 박뱅이라니...... 귀하다귀해 인터뷰 안 피하는 ...,긍정적,74.5,https://www.youtube.com/watch?v=PWil5aTn7e4,https://img.youtube.com/vi/PWil5aTn7e4/maxresd...
4,"데뷔 첫 완봉승, 천금의 한 점, 승리 직후의 모습 #애프터매치(08.27 키움전)",갑작스럽게 나오게 되었지만 늘 꼭 필요한 순간에 한방 터트려주는 안경 선배 멋있어요...,긍정적,53.0,https://www.youtube.com/watch?v=QkMRf7lvbe0,https://img.youtube.com/vi/QkMRf7lvbe0/maxresd...
5,✨삼성라이온즈✨ 평점 10/10 도파민 터지는 삼성라이온즈 야구를 추천하오. 소인은...,정빈이는 진짜 유명한 ㅈ댐감지기가 맞다 ㅋㅋㅋㅋㅋㅋㅋ진짜 벼랑끝에 왔을때 하나씩 해...,긍정적,36.5,https://www.youtube.com/watch?v=qergpNg580U,https://img.youtube.com/vi/qergpNg580U/maxresd...
6,"""인성이 여기서 나타나는 거야"" ""우리 지광이는요..평양냉면을 좋아해요"" T와 F적...",지찬아 너 삼성이랑 20년 계약했어… 기억 안 나 ? 태인아 너 비fa다년계약 20...,긍정적,21.5,https://www.youtube.com/watch?v=NjrZdLiEHYs,https://img.youtube.com/vi/NjrZdLiEHYs/maxresd...
7,(월간라클) 팬 vs 선수 라이온즈 푸른 피 TEST✏ #월간라클,2:12 좌승보다 지찬이가 형이라니...깜짝놀랐네.. 1:23 영웅이 협찬 해달라는...,긍정적,19.0,https://www.youtube.com/watch?v=BRnQTmWaV_A,https://img.youtube.com/vi/BRnQTmWaV_A/maxresd...


In [54]:
engine = create_engine(f'oracle+cx_oracle://{db_username}:{db_password}@{db_host}:{db_port}/?service_name={service_name}')
merged_df.to_sql(
    'emotion', 
    con=engine, 
    if_exists='append', 
    index=False, 
    dtype={'제목' : VARCHAR2(300),
        '댓글내용' : Text(),
        '감성분석' : VARCHAR2(50),
        '감성점수' : NUMBER,
        '링크' : VARCHAR2(225),
        '썸네일' : VARCHAR2(225)
    }
)
print("Table insert successfully")

Table insert successfully
